In [2]:
import pandas as pd
import nflgame
import numpy as np
import requests
from bs4 import BeautifulSoup

A Few notes: NFL game is unsupported for 2020 onwards, and does not give back years past 2009. However, this 10 year window, we can get an incredibly descriptive dataset. 

Each NFL season has 17 weeks. 

In [39]:
#generates Game DataFrame Instance for a specific game object
def generate_game_df(game_obj,week):
    stats_away = game_obj.stats_away
    stats_home = game_obj.stats_home
    score_string = ('|').join([x for x in game_obj.scores])
    schedule_stats = game_obj.schedule
    game_dict = {
        'game_eid': game_obj.eid,
        'home_team':game_obj.home,
        'away_team':game_obj.away,
        "away_first_downs":stats_away.first_downs,
        "away_passing_yds":stats_away.passing_yds,
        "away_penalty_count":stats_away.penalty_cnt,
        "away_penalty_yds":stats_away.penalty_yds,
        "away_posses_time_in_sec":stats_away.pos_time.total_seconds(),
        "away_punt_avg":stats_away.punt_avg,
        "away_punt_cnt":stats_away.punt_cnt,
        "away_punt_yds": stats_away.punt_yds,
        "away_rushing_yds": stats_away.rushing_yds,
        "away_total_yds":stats_away.total_yds,
        "turnovers":stats_away.turnovers,
        "scoring_string":score_string,
        "final_score": game_obj.nice_score(),
        "home_first_downs":stats_home.first_downs,
        "home_passing_yds":stats_home.passing_yds,
        "home_penalty_count":stats_home.penalty_cnt,
        "home_penalty_yds":stats_home.penalty_yds,
        "home_posses_time_in_sec":stats_home.pos_time.total_seconds(),
        "home_punt_avg":stats_home.punt_avg,
        "home_punt_cnt":stats_home.punt_cnt,
        "home_punt_yds": stats_home.punt_yds,
        "home_rushing_yds": stats_home.rushing_yds,
        "home_total_yds":stats_home.total_yds,
        "week":schedule_stats['week'],
        "season":game_obj.season(),
        "time_start": f"{schedule_stats['time']} : {schedule_stats['meridiem']}",
        'weekday':schedule_stats['wday'],
        'date': str(schedule_stats['month']) + '/'+str(schedule_stats['day']) + str(schedule_stats['year']),
        'players':(',').join([f"{c.player.name}:{c.team}" for c in game_obj.players if c.player != None]),
        'player_ids':(',').join([f"{c.player.playerid}:{c.team}" for c in game_obj.players if c.player != None])
    }
    return pd.DataFrame(game_dict,index=[1])

def generate_drive_df(drive_object):
    drive_dict = {
        'game_eid':drive_object.game.eid,
        'drive_num':drive_object.drive_num,
        'field_start':drive_object.field_start,
        'first_downs':drive_object.first_downs,
        'is_home':drive_object.home,
        'penalty_yds':drive_object.penalty_yds,
        'num_plays':drive_object.play_cnt,
        "possesion_time_in_sec":drive_object.pos_time.total_seconds(),
        "result":drive_object.result,
        'offense_team':drive_object.team,
        'quater':drive_object.time_end.quarter,
        'time_end':drive_object.time_end.clock,
        'time_start':drive_object.time_start.clock,
        'total_yrds':drive_object.total_yds
    }
    return pd.DataFrame(drive_dict,index=[1])

def generate_plays_df(play_obj):
    drive_id = play_obj.drive.game.eid + '-'+str(play_obj.drive.drive_num)+'-'+str(play_obj.drive.team)
    play_dict = {
        'play_description': play_obj.desc,
        'num_down': play_obj.down, #zero corresponds to kick off. 
        'drive_id': drive_id,
        'is_home': play_obj.home,
        'team':play_obj.team,
        'play_tag':play_obj.note,
        'play_id':drive_id+'-'+str(play_obj.playid),
        "scored_touchdown": play_obj.touchdown,
        'yards_togo':play_obj.yards_togo,
        'players_involved': ('|').join([f"{pl.name}:{pl.playerid}" for pl in play_obj.players])
    }
    return pd.DataFrame(play_dict,index=[1])

def generate_player_performance_df(plyr_obj,week,season,gm_eid):
    player_dict = {
        "player_name": plyr_obj.name,
        'player_id': plyr_obj.playerid,
        'team': plyr_obj.team,
        'game_eid':gm_eid,
        'week':week,
        'season':season,
        'defense_ast':None, 
        'defense_ffum':None, 
        'defense_int':None, 
        'defense_sk':None,
        'defense_tkl':None,
        'fumbles_lost':None, 
        'fumbles_rcv':None, 
        'fumbles_tot':None,
        'fumbles_trcv':None,
        'fumbles_yds':None,
        'kicking_fga':None, 
        'kicking_fgm':None,
        'kicking_fgyds':None, 
        'kicking_totpfg':None,
        'kicking_xpa':None,
        'kicking_xpb':None,
        'kicking_xpmade':None,
        'kicking_xpmissed':None,
        'kicking_xptot':None,
        'kickret_avg':None,
        'kickret_lng':None, 
        'kickret_lngtd':None, 
        'kickret_ret':None,
        'kickret_tds':None,
        'passing_att':None,
        'passing_cmp':None, 
        'passing_ints':None,
        'passing_tds':None, 
        'passing_twopta':None, 
        'passing_twoptm':None, 
        'passing_yds':None,
        'punting_avg':None,
        'punting_i20':None,
        'punting_lng':None,
        'punting_pts':None,
        'punting_yds':None, 
        'puntret_avg':None, 
        'puntret_lng':None, 
        'puntret_lngtd':None,
        'puntret_ret':None, 
        'puntret_tds':None,
        'receiving_lng':None,
        'receiving_lngtd':None,
        'receiving_rec':None, 
        'receiving_tds':None, 
        'receiving_twopta':None,
        'receiving_twoptm':None, 
        'receiving_yds':None, 
        'rushing_att':None, 
        'rushing_lng':None,
        'rushing_lngtd':None, 
        'rushing_tds':None, 
        'rushing_twopta':None, 
        'rushing_twoptm':None,
        'rushing_yds':None,   
    }
    for key,value in plyr_obj.stats.items():
        player_dict[key] = value
    return pd.DataFrame(player_dict,index=[1])
def generate_player_info_df(player_obj,week,season):
    player_not_stats_obj = player_obj.player
    if player_not_stats_obj == None:
        return
    else:
        player_info_dict ={
            "player_name": player_not_stats_obj.full_name,
            'player_id': player_obj.playerid,
            'team': player_obj.team,
            'week':week,
            'season':season,
            'weight':player_not_stats_obj.weight,
            'height':player_not_stats_obj.height,
            'college':player_not_stats_obj.college,
            'birthday':player_not_stats_obj.birthdate,
            'position':player_not_stats_obj.position,
            'years_pro':player_not_stats_obj.years_pro,
            'profile_url':player_not_stats_obj.profile_url

        }
    return pd.DataFrame(player_info_dict,index=[1])

In [40]:
#running with all of these true is a surefire way to burn your computers memory. 
def collect_dfs(play_gen=False,drive_gen=False,game_gen=False,player_info=False,player_performance=False):
    game_dfs = []
    play_dfs = []
    drive_dfs = []
    tmp = []
    player_dfs = []
    player_tmps = []
    player_info_dfs =[]
    for season in range(2009, 2020):
        print(f'collecting data for {season} season.')
        for wk in range(1,18):
            game = nflgame.games(season, week=wk)
            for gm in game:
                if player_performance or player_info:
                    player_tmps.append((gm.players,wk,season,gm.eid))
                if drive_gen:
                    drive_dfs.extend([generate_drive_df(drive) for drive in gm.drives])
                if play_gen:
                    tmp.extend(gm.drives)
            if game_gen:
                game_dfs.extend([generate_game_df(gm,wk) for gm in game])
    if play_gen:
        print('generating play dataset')
        for drive in tmp:
            for play in drive.plays:
                play_dfs.append(generate_plays_df(play))
    if player_performance:
        print('generating player performance dataset')

        for players,week,season,gm_eid in player_tmps:
            for player in players:
                player_dfs.append(generate_player_performance_df(player,week,season,gm_eid))
    if player_info:
        print('generating player info dataset')

        for players,week,season,gm_eid in player_tmps:
            for player in players:
                player_info_dfs.append(generate_player_info_df(player,week,season))
        
            
    print('concat-ing dataframes.')
    if game_gen:
        game_df =  pd.concat(game_dfs,ignore_index=True)
        game_df.index = range(game_df.shape[0])
        game_df.to_csv('all_game_data.csv')
    if drive_gen:
        driveDF = pd.concat(drive_dfs,ignore_index=True)
        driveDF.index = range(driveDF.shape[0])
        driveDF['drive_id'] = [driveDF['game_eid'].iloc[x] + '-'+str(driveDF['drive_num'].iloc[x]) + "-"+str(driveDF['offense_team'].iloc[x]) for x in driveDF.index ]
        driveDF.to_csv('all_drive_data.csv')
    if play_gen:
        playDF = pd.concat(play_dfs,ignore_index=True)
        playDF.index = range(playDF.shape[0])
        playDF.to_csv("all_play_data.csv")
    if player_performance:
        player_perf_df = pd.concat(player_dfs,ignore_index=True)
        player_perf_df.index = range(player_perf_df.shape[0])
        player_perf_df.to_csv("all_player_performance_data.csv")
    if player_info:
        player_inf_df = pd.concat(player_info_dfs,ignore_index=True)
        player_inf_df.index = range(player_inf_df.shape[0])
        player_inf_df.to_csv('all_personal_player_info_data.csv')

    return 

In [114]:
#game = nflgame.games(2017, week=3)
d=[]
l=[c for c in game[1].players]
for i in range(len(l)):
    d.append(l[i].stats.keys())
l=[]
for lis in d:
    l.append([str(c) for c in lis])
    

In [41]:
playDF = collect_dfs(game_gen=True,player_info=False, player_performance=True)

collecting data for 2009 season.
collecting data for 2010 season.
collecting data for 2011 season.
collecting data for 2012 season.
collecting data for 2013 season.
collecting data for 2014 season.
collecting data for 2015 season.
collecting data for 2016 season.
collecting data for 2017 season.
collecting data for 2018 season.
collecting data for 2019 season.
generating player performance dataset
generating player info dataset
concat-ing dataframes.


### [(str(c),c.team) for c in game_stats[0].players]
game_dfs

In [42]:
pl = pd.read_csv('all_game_data.csv',index_col=0)

In [43]:
pl.iloc[-1]

game_eid                                                          2019122915
home_team                                                                SEA
away_team                                                                 SF
away_first_downs                                                          18
away_passing_yds                                                         270
away_penalty_count                                                         8
away_penalty_yds                                                          57
away_posses_time_in_sec                                                 1624
away_punt_avg                                                             39
away_punt_cnt                                                              2
away_punt_yds                                                             94
away_rushing_yds                                                         128
away_total_yds                                                           398

In [62]:
L= []
for i in l:
    L.extend(i)
y=pd.Series(L).unique()

In [15]:
game_data = pd.read_csv('all_game_data.csv',index_col=0)

In [14]:
player_pers = pd.read_csv('all_player_performance_data.csv',index_col=0)

In [3]:
player_inf = pd.read_csv('all_personal_player_info_data.csv',index_col=0)

# Purge all nans from positions:

In [9]:
def get_position_from_profile_url(player_name,profile_url):
    html = requests.get(profile_url)
    soup = BeautifulSoup(html.text,'html.parser')
    res = soup.find_all('span', {"class":"nfl-c-player-header__position"})
    if len(res)> 0:
        return res[0].text.strip()
    else:
        #look for it in retired players. First, we execute a query request.
        query_url = f'https://www.nfl.com/players/retired/all?query={player_name}'
            html = requests.get(query_url)
            soup = BeautifulSoup(html.text,'html.parser')
        res = soup.find_all('a', {"class":"d3-o-player-fullname nfl-o-cta--link"})
        if len(res)>0:
            new_profile_url = 'https://www.nfl.com'+res[0]['href']
            get_position_from_profile_url(player_name,new_profile_url)

In [5]:
targets = player_inf[player_inf['position'].isna()]


In [6]:
#need to get all unique players and urls
name_urls = targets.loc[:,('player_name','profile_url')]
uq_urls = name_urls['profile_url'].unique()

args_for_retry = [name_urls[name_urls['profile_url'] == uq_urls[i]].iloc[0] for i in range(uq_urls.shape[0])]

In [7]:
uq_positions_names_urls = []
def scrape_retired_player_positions(uq_positions_names_urls,args_for_retry):
    for idx,value in enumerate(args_for_retry):
        name,url = value
        print(f'progress:{idx/len(args_for_retry)}')
        position = get_position_from_profile_url(name,url)
        uq_positions_names_urls.append([name,url,position])
    return uq_positions_names_urls

In [10]:
uq_positions_names_urls = scrape_retired_player_positions(uq_positions_names_urls,args_for_retry)
for triple in uq_positions_names_urls:
    player_inf.loc[player_inf.profile_url == triple[1],'position'] = triple[2]
player_inf.to_csv("all_personal_player_info_data.csv")

progress:0.0
progress:0.000282326369282891
progress:0.000564652738565782
progress:0.0008469791078486731
progress:0.001129305477131564
progress:0.001411631846414455
progress:0.0016939582156973462
progress:0.001976284584980237
progress:0.002258610954263128
progress:0.002540937323546019
progress:0.00282326369282891
progress:0.003105590062111801
progress:0.0033879164313946925
progress:0.0036702428006775835
progress:0.003952569169960474
progress:0.004234895539243365
progress:0.004517221908526256
progress:0.004799548277809147
progress:0.005081874647092038
progress:0.005364201016374929
progress:0.00564652738565782
progress:0.005928853754940711
progress:0.006211180124223602
progress:0.006493506493506494
progress:0.006775832862789385
progress:0.007058159232072276
progress:0.007340485601355167
progress:0.007622811970638058
progress:0.007905138339920948
progress:0.00818746470920384
progress:0.00846979107848673
progress:0.008752117447769622
progress:0.009034443817052512
progress:0.0093167701863354

progress:0.07876905702992659
progress:0.07905138339920949
progress:0.07933370976849238
progress:0.07961603613777526
progress:0.07989836250705816
progress:0.08018068887634106
progress:0.08046301524562394
progress:0.08074534161490683
progress:0.08102766798418973
progress:0.08130999435347261
progress:0.0815923207227555
progress:0.0818746470920384
progress:0.08215697346132128
progress:0.08243929983060418
progress:0.08272162619988707
progress:0.08300395256916997
progress:0.08328627893845285
progress:0.08356860530773574
progress:0.08385093167701864
progress:0.08413325804630152
progress:0.08441558441558442
progress:0.08469791078486731
progress:0.08498023715415019
progress:0.08526256352343309
progress:0.08554488989271598
progress:0.08582721626199886
progress:0.08610954263128176
progress:0.08639186900056466
progress:0.08667419536984754
progress:0.08695652173913043
progress:0.08723884810841333
progress:0.08752117447769622
progress:0.0878035008469791
progress:0.088085827216262
progress:0.08836815

progress:0.15923207227555053
progress:0.15951439864483344
progress:0.15979672501411632
progress:0.1600790513833992
progress:0.1603613777526821
progress:0.160643704121965
progress:0.16092603049124787
progress:0.16120835686053078
progress:0.16149068322981366
progress:0.16177300959909655
progress:0.16205533596837945
progress:0.16233766233766234
progress:0.16261998870694522
progress:0.16290231507622813
progress:0.163184641445511
progress:0.1634669678147939
progress:0.1637492941840768
progress:0.16403162055335968
progress:0.16431394692264256
progress:0.16459627329192547
progress:0.16487859966120835
progress:0.16516092603049123
progress:0.16544325239977414
progress:0.16572557876905702
progress:0.16600790513833993
progress:0.16629023150762282
progress:0.1665725578769057
progress:0.1668548842461886
progress:0.1671372106154715
progress:0.16741953698475437
progress:0.16770186335403728
progress:0.16798418972332016
progress:0.16826651609260304
progress:0.16854884246188595
progress:0.16883116883116

progress:0.23997741389045738
progress:0.24025974025974026
progress:0.24054206662902314
progress:0.24082439299830605
progress:0.24110671936758893
progress:0.2413890457368718
progress:0.24167137210615472
progress:0.2419536984754376
progress:0.2422360248447205
progress:0.2425183512140034
progress:0.24280067758328627
progress:0.24308300395256918
progress:0.24336533032185206
progress:0.24364765669113495
progress:0.24392998306041785
progress:0.24421230942970074
progress:0.24449463579898362
progress:0.24477696216826653
progress:0.2450592885375494
progress:0.2453416149068323
progress:0.2456239412761152
progress:0.24590626764539808
progress:0.24618859401468096
progress:0.24647092038396387
progress:0.24675324675324675
progress:0.24703557312252963
progress:0.24731789949181254
progress:0.24760022586109542
progress:0.2478825522303783
progress:0.24816487859966121
progress:0.2484472049689441
progress:0.24872953133822698
progress:0.2490118577075099
progress:0.24929418407679277
progress:0.2495765104460

progress:0.32156973461321287
progress:0.32185206098249575
progress:0.3221343873517787
progress:0.32241671372106157
progress:0.32269904009034445
progress:0.32298136645962733
progress:0.3232636928289102
progress:0.3235460191981931
progress:0.32382834556747603
progress:0.3241106719367589
progress:0.3243929983060418
progress:0.3246753246753247
progress:0.32495765104460755
progress:0.32523997741389044
progress:0.3255223037831734
progress:0.32580463015245625
progress:0.32608695652173914
progress:0.326369282891022
progress:0.3266516092603049
progress:0.3269339356295878
progress:0.3272162619988707
progress:0.3274985883681536
progress:0.3277809147374365
progress:0.32806324110671936
progress:0.32834556747600224
progress:0.3286278938452851
progress:0.32891022021456806
progress:0.32919254658385094
progress:0.3294748729531338
progress:0.3297571993224167
progress:0.3300395256916996
progress:0.33032185206098247
progress:0.3306041784302654
progress:0.3308865047995483
progress:0.33116883116883117
progr

...

progress:0.6507622811970638
progress:0.6510446075663467
progress:0.6513269339356296
progress:0.6516092603049125
progress:0.6518915866741953
progress:0.6521739130434783
progress:0.6524562394127611
progress:0.652738565782044
progress:0.653020892151327
progress:0.6533032185206098
progress:0.6535855448898927
progress:0.6538678712591756
progress:0.6541501976284585
progress:0.6544325239977414
progress:0.6547148503670243
progress:0.6549971767363072
progress:0.65527950310559
progress:0.655561829474873
progress:0.6558441558441559
progress:0.6561264822134387
progress:0.6564088085827217
progress:0.6566911349520045
progress:0.6569734613212874
progress:0.6572557876905702
progress:0.6575381140598532
progress:0.6578204404291361
progress:0.658102766798419
progress:0.6583850931677019
progress:0.6586674195369847
progress:0.6589497459062676
progress:0.6592320722755506
progress:0.6595143986448334
progress:0.6597967250141163
progress:0.6600790513833992
progress:0.6603613777526821
progress:0.660643704121964

progress:0.7337662337662337
progress:0.7340485601355167
progress:0.7343308865047995
progress:0.7346132128740824
progress:0.7348955392433654
progress:0.7351778656126482
progress:0.7354601919819311
progress:0.735742518351214
progress:0.7360248447204969
progress:0.7363071710897798
progress:0.7365894974590627
progress:0.7368718238283456
progress:0.7371541501976284
progress:0.7374364765669114
progress:0.7377188029361943
progress:0.7380011293054771
progress:0.7382834556747601
progress:0.7385657820440429
progress:0.7388481084133258
progress:0.7391304347826086
progress:0.7394127611518916
progress:0.7396950875211745
progress:0.7399774138904573
progress:0.7402597402597403
progress:0.7405420666290231
progress:0.740824392998306
progress:0.741106719367589
progress:0.7413890457368718
progress:0.7416713721061547
progress:0.7419536984754376
progress:0.7422360248447205
progress:0.7425183512140033
progress:0.7428006775832863
progress:0.7430830039525692
progress:0.743365330321852
progress:0.7436476566911

progress:0.8167701863354038
progress:0.8170525127046866
progress:0.8173348390739695
progress:0.8176171654432524
progress:0.8178994918125353
progress:0.8181818181818182
progress:0.8184641445511011
progress:0.818746470920384
progress:0.8190287972896668
progress:0.8193111236589498
progress:0.8195934500282326
progress:0.8198757763975155
progress:0.8201581027667985
progress:0.8204404291360813
progress:0.8207227555053642
progress:0.821005081874647
progress:0.82128740824393
progress:0.8215697346132129
progress:0.8218520609824957
progress:0.8221343873517787
progress:0.8224167137210615
progress:0.8226990400903444
progress:0.8229813664596274
progress:0.8232636928289102
progress:0.8235460191981931
progress:0.823828345567476
progress:0.8241106719367589
progress:0.8243929983060417
progress:0.8246753246753247
progress:0.8249576510446076
progress:0.8252399774138904
progress:0.8255223037831734
progress:0.8258046301524562
progress:0.8260869565217391
progress:0.8263692828910221
progress:0.82665160926030

progress:0.8997741389045737
progress:0.9000564652738566
progress:0.9003387916431395
progress:0.9006211180124224
progress:0.9009034443817052
progress:0.9011857707509882
progress:0.901468097120271
progress:0.9017504234895539
progress:0.9020327498588369
progress:0.9023150762281197
progress:0.9025974025974026
progress:0.9028797289666854
progress:0.9031620553359684
progress:0.9034443817052513
progress:0.9037267080745341
progress:0.9040090344438171
progress:0.9042913608130999
progress:0.9045736871823828
progress:0.9048560135516657
progress:0.9051383399209486
progress:0.9054206662902315
progress:0.9057029926595144
progress:0.9059853190287973
progress:0.9062676453980801
progress:0.9065499717673631
progress:0.906832298136646
progress:0.9071146245059288
progress:0.9073969508752118
progress:0.9076792772444946
progress:0.9079616036137775
progress:0.9082439299830605
progress:0.9085262563523433
progress:0.9088085827216262
progress:0.9090909090909091
progress:0.909373235460192
progress:0.909655561829

progress:0.9827780914737436
progress:0.9830604178430266
progress:0.9833427442123094
progress:0.9836250705815923
progress:0.9839073969508753
progress:0.9841897233201581
progress:0.984472049689441
progress:0.9847543760587238
progress:0.9850367024280068
progress:0.9853190287972897
progress:0.9856013551665725
progress:0.9858836815358555
progress:0.9861660079051383
progress:0.9864483342744212
progress:0.9867306606437041
progress:0.987012987012987
progress:0.98729531338227
progress:0.9875776397515528
progress:0.9878599661208357
progress:0.9881422924901185
progress:0.9884246188594015
progress:0.9887069452286844
progress:0.9889892715979672
progress:0.9892715979672502
progress:0.989553924336533
progress:0.9898362507058159
progress:0.9901185770750988
progress:0.9904009034443817
progress:0.9906832298136646
progress:0.9909655561829475
progress:0.9912478825522304
progress:0.9915302089215132
progress:0.9918125352907962
progress:0.9920948616600791
progress:0.9923771880293619
progress:0.99265951439864

In [56]:
player_pers.loc[player_pers.player_id == '00-0023252']['kicking_fgm']

749       2.0
1483      1.0
2307      1.0
2604      2.0
4826      0.0
         ... 
149993    1.0
151104    2.0
152295    3.0
152573    2.0
154201    2.0
Name: kicking_fgm, Length: 159, dtype: float64

In [111]:
player_pers.fillna(0)

,player_name,player_id,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,defense_sk,...,receiving_twopta,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds
0,B.Roethlisberger,00-0022924,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0
1,W.Parker,00-0022250,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,13.0,8.0,0.0,0.0,0.0,0.0,19.0
2,M.Moore,00-0022217,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,0.0,0.0,28.0,5.0,4.0,0.0,0.0,0.0,0.0,8.0
3,R.Mendenhall,00-0026163,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,6.0
4,S.Holmes,00-0024240,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,0.0,0.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154215,S.Thomas,00-0033870,SF,2019122915,17,2019,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154216,S.Day,00-0033010,SF,2019122915,17,2019,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154217,K.Givens,00-0035026,SF,2019122915,17,2019,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154218,A.Armstead,00-0032164,SF,2019122915,17,2019,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
get_position_from_profile_url(url)

In [124]:
game_data[game_data["game_eid"]==2009091000]

,game_eid,home_team,away_team,away_first_downs,away_passing_yds,away_penalty_count,away_penalty_yds,away_posses_time_in_sec,away_punt_avg,away_punt_cnt,...,home_punt_yds,home_rushing_yds,home_total_yds,week,season,time_start,weekday,date,players,player_ids
0,2009091000,PIT,TEN,18,234,5,28,1701,42,5,...,299,36,357,1,2009,8:30 : PM,Thu,9/102009,"Ben Roethlisberger:PIT,Willie Parker:PIT,Mewel...","00-0022924:PIT,00-0022250:PIT,00-0022217:PIT,0..."


In [43]:
player_name='Santonio Holmes'
query_url = f'https://www.nfl.com/players/retired/all?query={player_name}'
html = requests.get(query_url)
soup = BeautifulSoup(html.text,'html.parser')
res = soup.find_all('a', {"class":"d3-o-player-fullname nfl-o-cta--link"},href=True)

progress:0.0
progress:0.000282326369282891
progress:0.000564652738565782
progress:0.0008469791078486731
progress:0.001129305477131564
progress:0.001411631846414455
progress:0.0016939582156973462
progress:0.001976284584980237
progress:0.002258610954263128
progress:0.002540937323546019
progress:0.00282326369282891
progress:0.003105590062111801
progress:0.0033879164313946925
progress:0.0036702428006775835
progress:0.003952569169960474
progress:0.004234895539243365
progress:0.004517221908526256
progress:0.004799548277809147
progress:0.005081874647092038
progress:0.005364201016374929
progress:0.00564652738565782
progress:0.005928853754940711
progress:0.006211180124223602
progress:0.006493506493506494
progress:0.006775832862789385
progress:0.007058159232072276
progress:0.007340485601355167
progress:0.007622811970638058
progress:0.007905138339920948
progress:0.00818746470920384
progress:0.00846979107848673
progress:0.008752117447769622
progress:0.009034443817052512
progress:0.0093167701863354

progress:0.07876905702992659
progress:0.07905138339920949
progress:0.07933370976849238
progress:0.07961603613777526
progress:0.07989836250705816
progress:0.08018068887634106
progress:0.08046301524562394
progress:0.08074534161490683
progress:0.08102766798418973
progress:0.08130999435347261
progress:0.0815923207227555
progress:0.0818746470920384
progress:0.08215697346132128
progress:0.08243929983060418
progress:0.08272162619988707
progress:0.08300395256916997
progress:0.08328627893845285
progress:0.08356860530773574
progress:0.08385093167701864
progress:0.08413325804630152
progress:0.08441558441558442
progress:0.08469791078486731
progress:0.08498023715415019
progress:0.08526256352343309
progress:0.08554488989271598
progress:0.08582721626199886
progress:0.08610954263128176
progress:0.08639186900056466
progress:0.08667419536984754
progress:0.08695652173913043
progress:0.08723884810841333
progress:0.08752117447769622
progress:0.0878035008469791
progress:0.088085827216262
progress:0.08836815

progress:0.15923207227555053
progress:0.15951439864483344
progress:0.15979672501411632
progress:0.1600790513833992
progress:0.1603613777526821
progress:0.160643704121965
progress:0.16092603049124787
progress:0.16120835686053078
progress:0.16149068322981366
progress:0.16177300959909655
progress:0.16205533596837945
progress:0.16233766233766234
progress:0.16261998870694522
progress:0.16290231507622813
progress:0.163184641445511
progress:0.1634669678147939
progress:0.1637492941840768
progress:0.16403162055335968
progress:0.16431394692264256
progress:0.16459627329192547
progress:0.16487859966120835
progress:0.16516092603049123
progress:0.16544325239977414
progress:0.16572557876905702
progress:0.16600790513833993
progress:0.16629023150762282
progress:0.1665725578769057
progress:0.1668548842461886
progress:0.1671372106154715
progress:0.16741953698475437
progress:0.16770186335403728
progress:0.16798418972332016
progress:0.16826651609260304
progress:0.16854884246188595
progress:0.16883116883116

progress:0.23997741389045738
progress:0.24025974025974026
progress:0.24054206662902314
progress:0.24082439299830605
progress:0.24110671936758893
progress:0.2413890457368718
progress:0.24167137210615472
progress:0.2419536984754376
progress:0.2422360248447205
progress:0.2425183512140034
progress:0.24280067758328627
progress:0.24308300395256918
progress:0.24336533032185206
progress:0.24364765669113495
progress:0.24392998306041785
progress:0.24421230942970074
progress:0.24449463579898362
progress:0.24477696216826653
progress:0.2450592885375494
progress:0.2453416149068323
progress:0.2456239412761152
progress:0.24590626764539808
progress:0.24618859401468096
progress:0.24647092038396387
progress:0.24675324675324675
progress:0.24703557312252963
progress:0.24731789949181254
progress:0.24760022586109542
progress:0.2478825522303783
progress:0.24816487859966121
progress:0.2484472049689441
progress:0.24872953133822698
progress:0.2490118577075099
progress:0.24929418407679277
progress:0.2495765104460

progress:0.32156973461321287
progress:0.32185206098249575
progress:0.3221343873517787
progress:0.32241671372106157
progress:0.32269904009034445
progress:0.32298136645962733
progress:0.3232636928289102
progress:0.3235460191981931
progress:0.32382834556747603
progress:0.3241106719367589
progress:0.3243929983060418
progress:0.3246753246753247
progress:0.32495765104460755
progress:0.32523997741389044
progress:0.3255223037831734
progress:0.32580463015245625
progress:0.32608695652173914
progress:0.326369282891022
progress:0.3266516092603049
progress:0.3269339356295878
progress:0.3272162619988707
progress:0.3274985883681536
progress:0.3277809147374365
progress:0.32806324110671936
progress:0.32834556747600224
progress:0.3286278938452851
progress:0.32891022021456806
progress:0.32919254658385094
progress:0.3294748729531338
progress:0.3297571993224167
progress:0.3300395256916996
progress:0.33032185206098247
progress:0.3306041784302654
progress:0.3308865047995483
progress:0.33116883116883117
progr

...

progress:0.6507622811970638
progress:0.6510446075663467
progress:0.6513269339356296
progress:0.6516092603049125
progress:0.6518915866741953
progress:0.6521739130434783
progress:0.6524562394127611
progress:0.652738565782044
progress:0.653020892151327
progress:0.6533032185206098
progress:0.6535855448898927
progress:0.6538678712591756
progress:0.6541501976284585
progress:0.6544325239977414
progress:0.6547148503670243
progress:0.6549971767363072
progress:0.65527950310559
progress:0.655561829474873
progress:0.6558441558441559
progress:0.6561264822134387
progress:0.6564088085827217
progress:0.6566911349520045
progress:0.6569734613212874
progress:0.6572557876905702
progress:0.6575381140598532
progress:0.6578204404291361
progress:0.658102766798419
progress:0.6583850931677019
progress:0.6586674195369847
progress:0.6589497459062676
progress:0.6592320722755506
progress:0.6595143986448334
progress:0.6597967250141163
progress:0.6600790513833992
progress:0.6603613777526821
progress:0.660643704121964

progress:0.7337662337662337
progress:0.7340485601355167
progress:0.7343308865047995
progress:0.7346132128740824
progress:0.7348955392433654
progress:0.7351778656126482
progress:0.7354601919819311
progress:0.735742518351214
progress:0.7360248447204969
progress:0.7363071710897798
progress:0.7365894974590627
progress:0.7368718238283456
progress:0.7371541501976284
progress:0.7374364765669114
progress:0.7377188029361943
progress:0.7380011293054771
progress:0.7382834556747601
progress:0.7385657820440429
progress:0.7388481084133258
progress:0.7391304347826086
progress:0.7394127611518916
progress:0.7396950875211745
progress:0.7399774138904573
progress:0.7402597402597403
progress:0.7405420666290231
progress:0.740824392998306
progress:0.741106719367589
progress:0.7413890457368718
progress:0.7416713721061547
progress:0.7419536984754376
progress:0.7422360248447205
progress:0.7425183512140033
progress:0.7428006775832863
progress:0.7430830039525692
progress:0.743365330321852
progress:0.7436476566911

progress:0.8167701863354038
progress:0.8170525127046866
progress:0.8173348390739695
progress:0.8176171654432524
progress:0.8178994918125353
progress:0.8181818181818182
progress:0.8184641445511011
progress:0.818746470920384
progress:0.8190287972896668
progress:0.8193111236589498
progress:0.8195934500282326
progress:0.8198757763975155
progress:0.8201581027667985
progress:0.8204404291360813
progress:0.8207227555053642
progress:0.821005081874647
progress:0.82128740824393
progress:0.8215697346132129
progress:0.8218520609824957
progress:0.8221343873517787
progress:0.8224167137210615
progress:0.8226990400903444
progress:0.8229813664596274
progress:0.8232636928289102
progress:0.8235460191981931
progress:0.823828345567476
progress:0.8241106719367589
progress:0.8243929983060417
progress:0.8246753246753247
progress:0.8249576510446076
progress:0.8252399774138904
progress:0.8255223037831734
progress:0.8258046301524562
progress:0.8260869565217391
progress:0.8263692828910221
progress:0.82665160926030

progress:0.8997741389045737
progress:0.9000564652738566
progress:0.9003387916431395
progress:0.9006211180124224
progress:0.9009034443817052
progress:0.9011857707509882
progress:0.901468097120271
progress:0.9017504234895539
progress:0.9020327498588369
progress:0.9023150762281197
progress:0.9025974025974026
progress:0.9028797289666854
progress:0.9031620553359684
progress:0.9034443817052513
progress:0.9037267080745341
progress:0.9040090344438171
progress:0.9042913608130999
progress:0.9045736871823828
progress:0.9048560135516657
progress:0.9051383399209486
progress:0.9054206662902315
progress:0.9057029926595144
progress:0.9059853190287973
progress:0.9062676453980801
progress:0.9065499717673631
progress:0.906832298136646
progress:0.9071146245059288
progress:0.9073969508752118
progress:0.9076792772444946
progress:0.9079616036137775
progress:0.9082439299830605
progress:0.9085262563523433
progress:0.9088085827216262
progress:0.9090909090909091
progress:0.909373235460192
progress:0.909655561829

progress:0.9827780914737436
progress:0.9830604178430266
progress:0.9833427442123094
progress:0.9836250705815923
progress:0.9839073969508753
progress:0.9841897233201581
progress:0.984472049689441
progress:0.9847543760587238
progress:0.9850367024280068
progress:0.9853190287972897
progress:0.9856013551665725
progress:0.9858836815358555
progress:0.9861660079051383
progress:0.9864483342744212
progress:0.9867306606437041
progress:0.987012987012987
progress:0.98729531338227
progress:0.9875776397515528
progress:0.9878599661208357
progress:0.9881422924901185
progress:0.9884246188594015
progress:0.9887069452286844
progress:0.9889892715979672
progress:0.9892715979672502
progress:0.989553924336533
progress:0.9898362507058159
progress:0.9901185770750988
progress:0.9904009034443817
progress:0.9906832298136646
progress:0.9909655561829475
progress:0.9912478825522304
progress:0.9915302089215132
progress:0.9918125352907962
progress:0.9920948616600791
progress:0.9923771880293619
progress:0.99265951439864

In [31]:
name_urls["player_name"].unique().shape

(3508,)

In [192]:
test_players = [x for x in nflgame.games(2018)[0].players if x.player is not None and x.player.position == 'LB']

In [202]:
test_players[0].t

0

'defense_ast':None, 
        'defense_ffum':None, 
        'defense_int':None, 
        'defense_sk':None,
        'defense_tkl':None,
        'fumbles_lost':None, 
        'fumbles_rcv':None, 
        'fumbles_tot':None,
        'fumbles_trcv':None,
        'fumbles_yds':None,
        'kicking_fga':None, 
        'kicking_fgm':None,
        'kicking_fgyds':None, 
        'kicking_totpfg':None,
        'kicking_xpa':None,
        'kicking_xpb':None,
        'kicking_xpmade':None,
        'kicking_xpmissed':None,
        'kicking_xptot':None,
        'kickret_avg':None,
        'kickret_lng':None, 
        'kickret_lngtd':None, 
        'kickret_ret':None,
        'kickret_tds':None,
        'passing_att':None,
        'passing_cmp':None, 
        'passing_ints':None,
        'passing_tds':None, 
        'passing_twopta':None, 
        'passing_twoptm':None, 
        'passing_yds':None,
        'punting_avg':None,
        'punting_i20':None,
        'punting_lng':None,
        'punting_pts':None,
        'punting_yds':None, 
        'puntret_avg':None, 
        'puntret_lng':None, 
        'puntret_lngtd':None,
        'puntret_ret':None, 
        'puntret_tds':None,
        'receiving_lng':None,
        'receiving_lngtd':None,
        'receiving_rec':None, 
        'receiving_tds':None, 
        'receiving_twopta':None,
        'receiving_twoptm':None, 
        'receiving_yds':None, 
        'rushing_att':None, 
        'rushing_lng':None,
        'rushing_lngtd':None, 
        'rushing_tds':None, 
        'rushing_twopta':None, 
        'rushing_twoptm':None,
        'rushing_yds':None, 

In [22]:
player_inf.loc[player_inf.profile_url == test[1],'position'] = test[2]
player_inf.loc[player_inf.profile_url == test[1],'position']

3        RB
1501     RB
1831     RB
3251     RB
3543     RB
         ..
65710    RB
66659    RB
67501    RB
68425    RB
69307    RB
Name: position, Length: 68, dtype: object

In [80]:
def calculate_fanduel_offense_score(merged):
    x =   merged['rushing_yds']*0.1 + merged['rushing_tds']*6 + merged['passing_yds']*0.04 + merged['passing_tds']*4 + merged['passing_ints']*(-1) + merged['receiving_yds'] * 0.1 + merged['receiving_tds']*6 + merged['receiving_rec']*0.5 + merged['kickret_tds'] * 6 + merged['puntret_tds'] * 6 + merged['fumbles_lost'] * (-2) +merged['fumbles_trcv']*6 + merged['passing_twoptm']*2 + merged['passing_twopta'] * 2 +merged['kicking_xpmade'] * 1
    return x
def calculate_fanduel_kicker_score(concat):
    acc = []
    for i in range(concat.shape[0]):#iterate thru rows
        if concat.iloc[i]['kicking_fgyds'] < 40 and concat.iloc[i]['kicking_fgm'] > 1:
            acc.append(3)
        elif concat.iloc[i]['kicking_fgyds'] >= 40  and  concat.iloc[i]['kicking_fgyds'] < 50 and concat.iloc[i]['kicking_fgm'] > 1:
            acc.append(4)
        elif concat.iloc[i]['kicking_fgyds'] >= 50  and concat.iloc[i]['kicking_fgm'] > 1:
            acc.append(5)
        else:
            acc.append(0)
    return acc

#returns dictionary of teams and their scores
def parse_scoring_string(scoring_string):
    away_team,away_score,trash,home_team,home_score = scoring_string.split(' ')
    return {away_team:int(away_score[1:-1]), home_team:int(home_score[1:-1])}

def calculate_fanduel_def_score(concat,kicker_metadata_DF):
    concat = concat.fillna(0)
    player_spec_pts = concat['defense_sk']*1 + concat['defense_ffum']*2 + (concat['receiving_tds'] + concat['passing_tds'] + concat['rushing_tds'])*6 + concat['defense_int']*2 
    game_inf = pd.read_csv('all_game_data.csv',index_col=0)
    game_spec_acc = []
    for i in range(concat.shape[0]):
        team_curr_player = concat.iloc[i]['team'][0]
        curr_game = concat.iloc[i]['game_eid']
        if_home = team_curr_player == game_inf[game_inf['game_eid']==curr_game]['home_team'].iloc[0]  
        curr_game_score = parse_scoring_string(game_inf[game_inf['game_eid']==curr_game]['final_score'].iloc[0])
        kickers_in_test_game = kicker_metadata_DF.loc[kicker_metadata_DF.game_eid == curr_game, :].query('position == "K"')
        #calculate field goals first. Then, scores allowed. 
        # we hunt for kickers because we need to know how many field goals were blocked. 
        if not if_home:
            opp = game_inf[game_inf['game_eid']==curr_game]['home_team'].iloc[0]
            teams_kick = kickers_in_test_game['team']
            if teams_kick[teams_kick==opp].shape[0] == 0:
                block_val = 0
            else:
                target_idx_kick = teams_kick[teams_kick==opp].index[0]
                block_val = kickers_in_test_game.loc[target_idx_kick]['kicking_fga'] - kickers_in_test_game.loc[target_idx_kick]['kicking_fgm']
        #score calc:
        else:
            opp = game_inf[game_inf['game_eid']==curr_game]['away_team'].iloc[0]
            teams_kick = kickers_in_test_game['team']
            if teams_kick[teams_kick==opp].shape[0] == 0:
                block_val = 0
            else:
                target_idx_kick = teams_kick[teams_kick==opp].index[0]
                block_val = kickers_in_test_game.loc[target_idx_kick]['kicking_fga'] - kickers_in_test_game.loc[target_idx_kick]['kicking_fgm']       
        def_score_allowed = 0
        opp_score = curr_game_score[opp]
        if opp_score == 0:
            def_score_allowed+=10
        elif opp_score >= 1 and opp_score <= 6:
            def_score_allowed+=7
        elif opp_score >= 7 and opp_score<= 13:
            def_score_allowed+=4
        elif opp_score >= 14 and opp_score<= 20:
            def_score_allowed+=10
        elif opp_score >= 21 and opp_score<= 27:
            pass
        elif opp_score >= 28 and opp_score<= 34:
            def_score_allowed-=1
        else:
            def_score_allowed-=4
        #calc final fantasy score for this one defensive player in the game.
        final_def_score = def_score_allowed + block_val*2
        game_spec_acc.append(final_def_score)

    prior = pd.Series(game_spec_acc)
    player_spec_pts.index = range(player_spec_pts.shape[0])
    #be very careful with pandas series add, its pretty trash. 
    added_numpy_arr = prior.values + player_spec_pts.values
    return added_numpy_arr# (pd.Series(added_numpy_arr), (prior,player_spec_pts,added_numpy_arr))

In [84]:
def create_positional_dataframes(player_inf,player_pers,positions=['QB','WR','RB','TE'],to_csv=True):
    #fill nans so the score function doesnt get mad
    player_pers=player_pers.fillna(0)
    def_position_dataframes =[]
    for  position in positions:
        print(f"working on position df for: {position}")
        copied_positions = player_inf.loc[player_inf.position==position,:].copy()
        copied_positions_pruned = copied_positions.loc[:,('player_id','position')]
        copied_positions_pruned.index = range(copied_positions_pruned.shape[0])

        if position in ['QB','WR','RB','TE']:
            positional_collection = copied_positions_pruned.merge(player_pers,on='player_id')
            positional_collection.index = range(positional_collection.shape[0])
            positional_collection['fantasy_pts'] = calculate_fanduel_offense_score(positional_collection)
        if position == "K": #if kicker
            positional_collection = copied_positions_pruned.merge(player_pers,on='player_id')
            positional_collection.index = range(positional_collection.shape[0])
            positional_collection['fantasy_pts'] = calculate_fanduel_kicker_score(positional_collection)
        if position  in ['CB', 'DE', 'DT', 'LB','ILB', 'MLB','NT', 'OLB','SS','FS']:
            #generate kicker DF too. 
            copied_positions_kick = player_inf.loc[player_inf.position=="K",:].copy()
            copied_positions_kick_pruned = copied_positions_kick.loc[:,('player_id','position')]
            copied_positions_kick_pruned.index = range(copied_positions_kick_pruned.shape[0])
            kicker_positional_collection = copied_positions_kick_pruned.merge(player_pers,on='player_id')

            #then generate normal DF
            positional_collection = copied_positions_pruned.merge(player_pers,on='player_id')
            positional_collection.index = range(positional_collection.shape[0])
            positional_collection['fantasy_pts'] = calculate_fanduel_def_score(positional_collection,kicker_positional_collection)
            def_position_dataframes.append(positional_collection)
        if to_csv:
            if position in ['QB','WR','RB','TE',"K"]:
                positional_collection.to_csv(f"position_and_fantasy_pts/{position}_stats_and_fantasy_pts.csv")
    print('concatting DEF csv.')
    total_def_concat = pd.concat(def_position_dataframes,axis=0)
    total_def_concat.index = range(total_def_concat.shape[0])
    total_def_concat.to_csv('position_and_fantasy_pts/def_stats_and_fantasy_pts.csv')
    return positional_collection


In [55]:
#copyDF = player_inf.loc[player_inf.position=='LB',:].copy()


,player_name,player_id,team,week,season,weight,height,college,birthday,position,years_pro,profile_url
0,Ben Roethlisberger,00-0022924,PIT,1,2009,240,77,Miami (Ohio),3/2/1982,QB,17,http://www.nfl.com/player/benroethlisberger/25...
1,Willie Parker,00-0022250,PIT,1,2009,212,70,North Carolina,11/11/1980,RB,7,http://www.nfl.com/player/willieparker/2505709...
2,Mewelde Moore,00-0022217,PIT,1,2009,209,71,Tulane,7/24/1982,RB,9,http://www.nfl.com/player/meweldemoore/2505690...
3,Rashard Mendenhall,00-0026163,PIT,1,2009,225,70,Illinois,6/19/1987,RB,6,http://www.nfl.com/player/rashardmendenhall/93...
4,Santonio Holmes,00-0024240,PIT,1,2009,192,71,Ohio State,3/3/1984,WR,9,http://www.nfl.com/player/santonioholmes/25068...
...,...,...,...,...,...,...,...,...,...,...,...,...
153924,Emmanuel Moseley,00-0034096,SF,17,2019,190,71,Tennessee,3/25/1996,DB,3,http://www.nfl.com/player/emmanuelmoseley/2561...
153925,Solomon Thomas,00-0033870,SF,17,2019,280,75,Stanford,8/26/1995,DE,4,http://www.nfl.com/player/solomonthomas/255801...
153926,Sheldon Day,00-0033010,SF,17,2019,285,73,Notre Dame,7/1/1994,DT,5,http://www.nfl.com/player/sheldonday/2555241/p...
153927,Arik Armstead,00-0032164,SF,17,2019,290,79,Oregon,11/15/1993,DE,6,http://www.nfl.com/player/arikarmstead/2552493...


In [20]:
#player_inf.loc[player_inf.position =='K',:]
copied_positions_kick.drop_duplicates(inplace=True)
kicker_positional_collection = copied_positions_kick_pruned.merge(player_pers,on='player_id')
kicker_positional_collection.drop_duplicates()

,player_id,position,player_name,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,...,receiving_twopta,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds
0,00-0020737,K,J.Reed,PIT,2009091000,1,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00-0020737,K,J.Reed,PIT,2009092011,2,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00-0020737,K,J.Reed,PIT,2009092701,3,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-0020737,K,J.Reed,PIT,2009100412,4,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-0020737,K,J.Reed,PIT,2009101103,5,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548843,00-0035269,K,M.Gay,TB,2019120104,13,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548844,00-0035269,K,M.Gay,TB,2019120808,14,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548845,00-0035269,K,M.Gay,TB,2019121502,15,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548846,00-0035269,K,M.Gay,TB,2019122100,16,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
player_pers=player_pers.fillna(0)
# game eid : 2019122912
#metadata_df = pd.concat([player_inf,player_pers],join='inner',axis=1)
copied_positions = player_inf.loc[player_inf.position=='K',:].copy()
copied_positions_test = copied_positions.loc[:,('player_id','position')]
copied_positions_test.index = range(copied_positions_test.shape[0])
positional_collection = copied_positions_test.merge(player_pers,on='player_id')
positional_collection.index = range(positional_collection.shape[0])
positional_collection.drop_duplicates(inplace=True)
copied_positions_kick = player_inf.loc[player_inf.position=="K",:].copy()
copied_positions_kick_pruned = copied_positions_kick.loc[:,('player_id','position')]
copied_positions_kick_pruned.index = range(copied_positions_kick_pruned.shape[0])
kicker_positional_collection = copied_positions_kick_pruned.merge(player_pers,on='player_id')
kicker_positional_collection.drop_duplicates(inplace=True)

#kicker_positional_collection.drop_duplicates(inplace=True)

            
positional_collection['fantasy_pts'] = calculate_fanduel_kicker_score(positional_collection) 
#positional_collection[positional_collection['game_eid']==2019122912].query('position=="K"')

In [40]:
opp=game_data[game_data['game_eid']==2009101103]['home_team'].iloc[0]
kickers_in_test_game = kicker_positional_collection.loc[kicker_positional_collection.game_eid == 2009101103, :].query('position == "K"')
#kickers_in_test_game.drop_duplicates()
teams_kick_test = kickers_in_test_game['team']
idx = teams_kick_test[teams_kick_test==opp]#.index[0]
#positional_collection['game_eid'].isna().sum()
#teams_kick_test
#player_pers.loc[player_pers.game_eid==2010091911,:]
teams_kick_test
idx.shape
# check to see if any kicker exists, if so do the normal thing. else just do 0

(0,)

In [82]:
positional_collection

,player_id,position,player_name,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,...,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds,fantasy_pts
0,00-0020737,K,J.Reed,PIT,2009091000,1,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,00-0020737,K,J.Reed,PIT,2009092011,2,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,00-0020737,K,J.Reed,PIT,2009092701,3,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,00-0020737,K,J.Reed,PIT,2009100412,4,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,00-0020737,K,J.Reed,PIT,2009101103,5,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548843,00-0035269,K,M.Gay,TB,2019120104,13,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
548844,00-0035269,K,M.Gay,TB,2019120808,14,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
548845,00-0035269,K,M.Gay,TB,2019121502,15,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
548846,00-0035269,K,M.Gay,TB,2019122100,16,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [86]:
all_positions = ['QB','WR','RB','TE','K','CB', 'DE', 'DT', 'LB','ILB', 'MLB','NT', 'OLB','SS','FS']
create_positional_dataframes(player_inf,player_pers,all_positions)

working on position df for: QB
working on position df for: WR
working on position df for: RB
working on position df for: TE
working on position df for: K
working on position df for: CB
working on position df for: DE


KeyboardInterrupt: 

In [210]:
player_pers = player_pers.fillna(0)
concat = pd.concat([copyDF,player_pers],axis=1,join='inner')
concat['fantasy_pts']=calculate_fanduel_offense_score(concat)
test_game = concat.iloc[3]['game_eid']

In [10]:
game_inf = pd.read_csv('all_game_data.csv',index_col=0)
#look up game_eid, match it with the one in the player stats, and look at the players team.
#then, we look at the score of the opposing team, and from there, calculate fantasy scores. 

#look up game_eid, match it with the kickers and measure whether they had their field goals and kickers blocked. 

In [181]:
pd.concat([])

ValueError: No objects to concatenate

In [312]:
big_concat = pd.concat([player_pers,player_inf],join='inner',axis=1)


In [416]:
kickers_in_test_game = big_concat.loc[big_concat.game_eid == test_game, :].query('position == "K"')
kickers_in_test_game

,player_name,player_id,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,defense_sk,...,team,week,season,weight,height,college,birthday,position,years_pro,profile_url
9,J.Reed,00-0020737,PIT,2009091000,1,2009,0.0,0.0,0.0,0.0,...,PIT,1,2009,225,71,North Carolina,4/9/1979,K,10,http://www.nfl.com/player/jeffreed/2504853/pro...
37,R.Bironas,00-0020962,TEN,2009091000,1,2009,0.0,0.0,0.0,0.0,...,TEN,1,2009,208,72,Georgia Southern,1/29/1978,K,9,http://www.nfl.com/player/robbironas/2504934/p...


In [397]:
kickers_in_test_game['kicking_fga'] - kickers_in_test_game['kicking_fgm']

9     0.0
37    1.0
dtype: float64

In [426]:
current_team = concat.iloc[3]['team'][0]


player_name       Chris Gocong
player_id           00-0024286
team                       PHI
week                         1
season                    2009
                      ...     
rushing_tds                  0
rushing_twopta               0
rushing_twoptm               0
rushing_yds                  0
fantasy_pts                  0
Name: 209, Length: 74, dtype: object

In [417]:
if_home = current_team==game_inf[game_inf['game_eid']==test_game]['home_team'].iloc[0]

In [409]:
kickers_in_test_game = big_concat.loc[big_concat.game_eid == test_game, :].query('position == "K"')

In [421]:
kickers_in_test_game['kicking_fgm'][current_team_idx]

TypeError: must be str, not Series

In [430]:
opp = game_inf[game_inf['game_eid']==test_game]['away_team'].iloc[0]
teams_kick = kickers_in_test_game['team'].iloc[:,0]
target_idx_kick = teams_kick[teams_kick==opp].index[0]
kickers_in_test_game.loc[target_idx_kick]
block_val = kickers_in_test_game.loc[target_idx_kick]['kicking_fga'] - kickers_in_test_game.loc[target_idx_kick]['kicking_fgm']
block_val

1.0

In [536]:
copied_positions = player_inf.loc[player_inf.position=='LB',:].copy()
positional_collection = pd.concat([copied_positions,player_pers],axis=1,join='inner')
positional_collection.index= range(positional_collection.shape[0])
positional_collection['fantasy_pts']= calculate_fanduel_def_score(positional_collection,big_concat)

In [413]:
test_game = positional_collection.iloc[0]['game_eid']

In [503]:
positional_collection.shape

(7481, 74)

In [79]:
positional_collection

,player_id,position,player_name,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,...,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds,fantasy_pts
0,00-0020840,FS,R.Clark,PIT,2009091000,1,2009,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,00-0020840,FS,R.Clark,PIT,2009092011,2,2009,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2,00-0020840,FS,R.Clark,PIT,2009092701,3,2009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,00-0020840,FS,R.Clark,PIT,2009100412,4,2009,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0
4,00-0020840,FS,R.Clark,PIT,2009101103,5,2009,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440196,00-0034556,FS,K.Kelly,PIT,2019090812,1,2019,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
440197,00-0034556,FS,K.Kelly,PIT,2019091507,2,2019,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440198,00-0034556,FS,K.Kelly,PIT,2019100607,5,2019,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
440199,00-0034556,FS,K.Kelly,PIT,2019101311,6,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0


In [513]:
debug[1]+debug[2]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
153875   NaN
153895   NaN
153902   NaN
153914   NaN
153923   NaN
Length: 14629, dtype: float64

In [514]:
debug[1]

0       6.0
1       6.0
2       4.0
3       4.0
4       6.0
       ... 
7476    4.0
7477    4.0
7478    4.0
7479    4.0
7480    4.0
Length: 7481, dtype: float64

In [519]:
new_arr = debug[1].values + debug[2].values

In [529]:
debug[2].index = range(debug[2].shape[0])

94

In [7]:
kickers =pd.read_csv('K_stats_and_fantasy_pts.csv',index_col=0)

In [8]:
kickers.iloc[-1]['game_eid']

2019122910

In [22]:
game_inf

,game_eid,home_team,away_team,away_first_downs,away_passing_yds,away_penalty_count,away_penalty_yds,away_posses_time_in_sec,away_punt_avg,away_punt_cnt,...,home_punt_yds,home_rushing_yds,home_total_yds,week,season,time_start,weekday,date,players,player_ids
0,2009091000,PIT,TEN,18,234,5,28,1701,42,5,...,299,36,357,1,2009,8:30 : PM,Thu,9/102009,"Ben Roethlisberger:PIT,Willie Parker:PIT,Mewel...","00-0022924:PIT,00-0022250:PIT,00-0022217:PIT,0..."
1,2009091300,ATL,MIA,16,163,4,27,1747,39,5,...,155,68,281,1,2009,1:00 : PM,Sun,9/132009,"Matt Ryan:ATL,Michael Turner:ATL,Jerious Norwo...","00-0026143:ATL,00-0022821:ATL,00-0024294:ATL,0..."
2,2009091301,BAL,KC,11,159,3,15,1211,47,5,...,87,198,501,1,2009,1:00 : PM,Sun,9/132009,"Joe Flacco:BAL,Ray Rice:BAL,Willis McGahee:BAL...","00-0026158:BAL,00-0026195:BAL,00-0022178:BAL,0..."
3,2009091302,CAR,PHI,17,82,9,76,1897,35,4,...,223,86,169,1,2009,1:00 : PM,Sun,9/132009,"Jake Delhomme:CAR,Matt Moore:CAR,Josh McCown:C...","00-0004161:CAR,00-0025708:CAR,00-0021206:CAR,0..."
4,2009091303,CIN,DEN,10,227,6,39,1593,36,8,...,278,86,307,1,2009,1:00 : PM,Sun,9/132009,"Carson Palmer:CIN,Cedric Benson:CIN,Chad Johns...","00-0021429:CIN,00-0023439:CIN,00-0020397:CIN,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2019122911,NYG,PHI,25,279,7,55,1885,40,7,...,257,122,397,17,2019,4:25 : PM,Sun,12/292019,"Daniel Jones:NYG,Saquon Barkley:NYG,Javorius A...","00-0035710:NYG,00-0034844:NYG,00-0031577:NYG,0..."
2812,2019122912,TB,ATL,28,286,5,53,2182,35,2,...,120,136,329,17,2019,1:00 : PM,Sun,12/292019,"Jameis Winston:TB,Ronald Jones:TB,Peyton Barbe...","00-0031503:TB,00-0034816:TB,00-0032741:TB,00-0..."
2813,2019122913,DEN,OAK,18,380,10,107,1980,40,3,...,254,70,238,17,2019,4:25 : PM,Sun,12/292019,"Drew Lock:DEN,Phillip Lindsay:DEN,Royce Freema...","00-0035704:DEN,00-0034109:DEN,00-0034838:DEN,0..."
2814,2019122914,LA,ARI,22,322,8,63,1382,44,3,...,156,113,424,17,2019,4:25 : PM,Sun,12/292019,"Jared Goff:LA,Todd Gurley:LA,Malcolm Brown:LA,...","00-0033106:LA,00-0032241:LA,00-0031806:LA,00-0..."


In [16]:
kickers.iloc[-1].keys()

Index(['player_name', 'player_id', 'team', 'week', 'season', 'weight',
       'height', 'college', 'birthday', 'position', 'years_pro', 'profile_url',
       'player_name.1', 'player_id.1', 'team.1', 'game_eid', 'week.1',
       'season.1', 'defense_ast', 'defense_ffum', 'defense_int', 'defense_sk',
       'defense_tkl', 'fumbles_lost', 'fumbles_rcv', 'fumbles_tot',
       'fumbles_trcv', 'fumbles_yds', 'kicking_fga', 'kicking_fgm',
       'kicking_fgyds', 'kicking_totpfg', 'kicking_xpa', 'kicking_xpb',
       'kicking_xpmade', 'kicking_xpmissed', 'kicking_xptot', 'kickret_avg',
       'kickret_lng', 'kickret_lngtd', 'kickret_ret', 'kickret_tds',
       'passing_att', 'passing_cmp', 'passing_ints', 'passing_tds',
       'passing_twopta', 'passing_twoptm', 'passing_yds', 'punting_avg',
       'punting_i20', 'punting_lng', 'punting_pts', 'punting_yds',
       'puntret_avg', 'puntret_lng', 'puntret_lngtd', 'puntret_ret',
       'puntret_tds', 'receiving_lng', 'receiving_lngtd', 'receiving

In [24]:
kickers

,player_name,player_id,team,week,season,weight,height,college,birthday,position,...,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds,fantasy_pts
0,Jeff Reed,00-0020737,PIT,1,2009,225,71,North Carolina,4/9/1979,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,Rob Bironas,00-0020962,TEN,1,2009,208,72,Georgia Southern,1/29/1978,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Jason Elam,00-0004811,ATL,1,2009,195,71,Hawaii,3/8/1970,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,Dan Carpenter,00-0026003,MIA,1,2009,228,74,Montana,11/25/1985,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,Stephen Hauschka,00-0025944,BAL,1,2009,210,76,North Carolina State,6/29/1985,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463,Daniel Carlson,00-0034161,OAK,17,2019,215,77,Auburn,1/23/1995,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5464,Greg Zuerlein,00-0029621,LA,17,2019,191,72,Missouri Western State,12/27/1987,K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5465,Zane Gonzalez,00-0033862,ARI,17,2019,202,72,Arizona State,5/7/1995,K,...,0.0,0.0,23.0,14.0,14.0,1.0,0.0,0.0,113.0,0
5466,Jason Myers,00-0031492,SEA,17,2019,190,70,Marist,5/12/1991,K,...,0.0,6.0,6.0,14.0,0.0,0.0,0.0,0.0,48.0,0


In [26]:
qbs = pd.read_csv('./position_and_fantasy_pts/QB_stats_and_fantasy_pts.csv',index_col=0)

In [29]:
qbs.iloc[-1]['passing_tds']

0.0

In [31]:
qbs.iloc[-1]['passing_']

0.0

In [87]:
import os

In [91]:
for file in os.listdir('./position_and_fantasy_pts/'):
    x = pd.read_csv(f'./position_and_fantasy_pts/{file}',index_col=0)
    x = x.drop_duplicates()
    x.to_csv(f'./position_and_fantasy_pts/{file}')

/home/owleyes/anaconda3/envs/stat/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/owleyes/anaconda3/envs/stat/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (74) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [92]:
x

,player_id,position,player_name,team,game_eid,week,season,defense_ast,defense_ffum,defense_int,...,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds,fantasy_pts
0,00-0022924,QB,B.Roethlisberger,PIT,2009091000,1,2009,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,16.82
1,00-0022924,QB,B.Roethlisberger,PIT,2009092011,2,2009,0.0,0.0,0.0,...,0.0,0.0,3.0,5.0,0.0,1.0,0.0,0.0,6.0,18.44
2,00-0022924,QB,B.Roethlisberger,PIT,2009092701,3,2009,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,3.0,20.34
3,00-0022924,QB,B.Roethlisberger,PIT,2009100412,4,2009,0.0,0.0,0.0,...,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,3.0,27.62
4,00-0022924,QB,B.Roethlisberger,PIT,2009101103,5,2009,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,-3.0,21.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557974,00-0029857,QB,R.Griffin,TB,2019120808,14,2019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.72
557975,00-0029857,QB,R.Griffin,TB,2019121502,15,2019,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,-1.0,-0.10
557978,00-0035251,QB,W.Grier,CAR,2019122207,16,2019,0.0,0.0,0.0,...,0.0,0.0,4.0,11.0,0.0,0.0,0.0,0.0,17.0,7.66
557979,00-0035251,QB,W.Grier,CAR,2019122902,17,2019,0.0,0.0,0.0,...,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,5.0,-2.34
